In [ ]:
import socket
import pymongo

#socket.AF_INET => Using IPv4
#socket.SOCK_STREAM => Using TCP
listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#'127.0.0.1 loop back address, 2345 port number
listen_socket.bind(('127.0.0.1', 12345))
print("waiting connection...")
listen_socket.listen()
game_socket, addr = listen_socket.accept()
print("connection accepted:")

client = pymongo.MongoClient('127.0.0.1', 27017)
db = client.get_database('Supermarket')
col = db.get_collection('supermarket')
result = col.find({},{"_id":0, "item_name":1})
items = []
for doc in result:
    result = list(doc.values())
    items.append(result[0])
    
print(items)
client.close()
    
done = False
while not done:
    # server always waits for client's msg
    rx_msg_str = ""
    while "\n" not in rx_msg_str:
        rx_msg_str += game_socket.recv(1024).decode()
        
    if rx_msg_str[:5] == "CHECK":
        msg_check = "CHECK"
        header, action, item = rx_msg_str[:-1].split(',')
        client = pymongo.MongoClient('127.0.0.1', 27017)
        db = client.get_database('Supermarket')
        col = db.get_collection('supermarket')
        
        if action == "PRICE":
            msg_check += ",PRICE"
            # get price of all items named item
            query = {"item_name": str(item)}
            fields = {"_id":0, "price":1}
            
            # get price of item requested
            result = col.find(query,fields)
            client.close()
            
            result_lst = []
            # check if item is an existing item, then send it
            for doc in result:
                result_lst += list(doc.values())
            print(result_lst)
            if item in items:
                msg_check += "," + action + "," + item + "," + str(result_lst[0])
            msg_check += "\n"
            game_socket.send(msg_check.encode())
            
        elif action == "TYPE":
            msg_check += ",TYPE"
            #get item_name of all item type = item
            query = {"item_type": str(item)}
            fields = {"_id":0, "item_name":1}
            
            # get items requested
            result = col.find(query,fields)
            client.close()
        
            # get value of all item_name and append to result_lst 
            result_lst = []
            for doc in result:
                temp = list(doc.values())
                print(doc)
                print(temp)
                result_lst.append(temp[0])
            print(result_lst)
            msg_check += "," + action + "," + item + "," + str(result_lst) + "\n"
            game_socket.send(msg_check.encode())
        
        elif action == "DISC":
            msg_check += ",DISC"
            # get both price and discount
            query = {"item_name": str(item)}
            fields = {"_id":0, "price":1, "discount_in_percent":1}
            
            # get price and discount of item 
            result = col.find(query,fields)
            client.close()
                
            # calculate the final price
            price = 0.0
            for doc in result:
                temp = list(doc.values())
                
            price, disc = float(temp[0]), float(temp[1])
            price = (price / 100.0) * (100 - disc)
            price_round = "{:.2f}".format(price)
            msg_check += "," +  action + "," + item + "," + str(price_round) + "\n"
            game_socket.send(msg_check.encode())
            
    elif rx_msg_str[:6] == "UPDATE":
        msg_update = "UPDATE"
        header, item_name, price = rx_msg_str[:-1].split(',')
        
        query = {"item_name":str(item_name)}
        update = { "$set": {"price":float(price)}}
        
        if item_name in items:
            client = pymongo.MongoClient('127.0.0.1', 27017)
            db = client.get_database('Supermarket')
            col = db.get_collection('supermarket')
            col.update(query, update)
            client.close()
            msg_update += ",OK"
        else:
            msg_update += ",FAIL"
        msg_update += "\n"
        game_socket.send(msg_update.encode())
            
    elif rx_msg_str[:6] == "DELETE":
        msg_delete = "DELETE"
        header, item_name = rx_msg_str[:-1].split(',')
        
        query = {"item_name": item_name}
        
        if item_name in items:
            client = pymongo.MongoClient('127.0.0.1', 27017)
            db = client.get_database('Supermarket')
            col = db.get_collection('supermarket')
            col.remove(query)
            client.close()
            items.remove(item_name)
            msg_delete += ",OK"
        else:
            msg_delete += ",FAIL"
        msg_delete += "\n"
        game_socket.send(msg_delete.encode())
        
    elif rx_msg_str[:3] == "ADD":
        msg_add = "ADD"
        header, item_name, item_type, price, discount = rx_msg_str[:-1].split(',')
        dict = {"item_name": item_name,
                "item_type": item_type,
                "price": price,
                "discount_in_percent": discount}
        client = pymongo.MongoClient('127.0.0.1', 27017)
        db = client.get_database('Supermarket')
        col = db.get_collection('supermarket')
        col.insert_one(dict)
        client.close
        items.append(item_name)
        msg_add += ",OK\n"
        game_socket.send(msg_add.encode())
        
    elif rx_msg_str[:-1] == "QUIT":
        done = True
        game_socket.send("QUIT\n".encode())
        print("User quits.")
    else:
        game_socket.send("\n".encode())
        
game_socket.close()
listen_socket.close()